In [1]:
from wiktionaryparser import WiktionaryParser
import requests
from bs4 import BeautifulSoup as bs
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))  
import pandas as pd
df = pd.read_csv('/home/jackragless/projects/local/DAIC_glossary_generator/big_data/unigram_freq.csv')
df = list(df['word'])

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jackragless/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
parser = WiktionaryParser()
def wikt_extract(kw):
    extract = parser.fetch(kw,'english')
    temp_def = {}
    for i in extract[0]['definitions']:
        temp_pos = []
        for j in i['text'][1:]:
            if not (j[0]=='(' and j[-1]==')'):
                temp_pos.append(j)
        temp_def[i['partOfSpeech']] = temp_pos
    return temp_def

In [12]:
def definition_general_prepend(definition):
    for pos in definition.keys():
        for sent in range(len(definition[pos])):
            if definition[pos][sent][0]!='(':
                definition[pos][sent] = '(general) ' + definition[pos][sent]
    return definition

In [14]:
FINAL = {}
FAILED = []
count = 0
for word in df[10000:30000]:
    count += 1
    if word not in stop_words:
        try:
            FINAL[word] = definition_general_prepend(wikt_extract(word))
        except: 
            FAILED.append(word)
        print(count, end='\r')

In [50]:
pd.DataFrame(FINAL).to_csv('wikt_definition_mined_unigram.csv')